In [1]:
# jack tarricone
# february 18th, 2021 

# amended dnppy.radar.create_header script to work for current UAVSAR annotation file format
# it parses the annoation file by finding the line of your selected search phrase is in
# it then searches that line by characters, and creates an object out of that character selection
# does this for data type, lat, lon, lines, samples
# it then inserts these information into the .hdr file
# in future add pixel size as well
# as well as delinating between 4 (real), and 6 (complex) data type for .int

__author__ = ["Daniel Jensen, danieljohnjensen@gmail.com",
              "Scott Baron",
              "Jwely"]


In [34]:
import os
import re
import codecs
import rasterio
import numpy
import matplotlib
from matplotlib import pyplot
import glob
import subprocess
from osgeo import gdal

In [64]:
#folder is path to a folder with an .ann and .grd file
def uavsar_tiff_convert(folder):
    """
    Builds a header file for the input UAVSAR .grd file,
    allowing the data to be read as a raster dataset.
    :param folder:   the folder containing the UAVSAR .grd and .ann files
    """


    os.chdir(folder)

    # Empty lists to put information that will be recalled later.
    Lines_list = []
    Samples_list = []
    Latitude_list = []
    Longitude_list = []
    Files_list = []

    # Step 1: Look through folder and determine how many different flights there are
    # by looking at the HDR files.
    for files in os.listdir(folder):
        if files [-4:] == ".grd":
            newfile = open(files[0:-4] + ".hdr", 'w')
            newfile.write("""ENVI
description = {DESCFIELD}
samples = NSAMP
lines = NLINE
bands = 1
header offset = 0
data type = DATTYPE
interleave = bsq
sensor type = Unknown
byte order = 0
map info = {Geographic Lat/Lon, 
            1.000, 
            1.000, 
            LON, 
            LAT,  
            0.0000555600000000, 
            0.0000555600000000, 
            WGS-84, units=Degrees}
wavelength units = Unknown
                """
                          )
            newfile.close()
            if files[0:18] not in Files_list:
                Files_list.append(files[0:18])

    #Variables used to recall indexed values.
    var1 = 0

    #Step 2: Look through the folder and locate the annotation file(s).
    # These can be in either .txt or .ann file types.
    for files in os.listdir(folder):
        if Files_list[var1] and files[-4:] == ".txt" or files[-4:] == ".ann":
            #Step 3: Once located, find the info we are interested in and append it to
            # the appropriate list. We limit the variables to <=1 so that they only
            # return two values (one for each polarization of
            searchfile = codecs.open(files, encoding='windows-1252')
            for line in searchfile:
                if "Ground Range Data Latitude Lines" in line:
                    Lines = line[65:70]
                    print(Lines)
                    if Lines not in Lines_list:
                        Lines_list.append(Lines)

                elif "Ground Range Data Longitude Samples" in line:
                    Samples = line[65:70]
                    print(Samples)
                    if Samples not in Samples_list:
                        Samples_list.append(Samples)

                elif "Ground Range Data Starting Latitude" in line:
                    Latitude = line[65:85]
                    print(Latitude)
                    if Latitude not in Latitude_list:
                        Latitude_list.append(Latitude)

                elif "Ground Range Data Starting Longitude" in line:
                    Longitude = line[65:85]
                    print(Longitude)
                    if Longitude not in Longitude_list:
                        Longitude_list.append(Longitude)
                        
                 
            #Reset the variables to zero for each different flight date.
            var1 = 0
            searchfile.close()


    # Step 3: Open HDR file and replace data.
    for files in os.listdir(folder):
        if files[-4:] == ".hdr":
            with open(files, "r") as sources:
                lines = sources.readlines()
            with open(files, "w") as sources:
                for line in lines:
                    if "data type = DATTYPE" in line:
                        sources.write(re.sub(line[12:19], "4", line))
                    elif "DESCFIELD" in line:
                        sources.write(re.sub(line[15:24], folder, line))
                    elif "lines" in line:
                        sources.write(re.sub(line[8:13], Lines, line))
                    elif "samples" in line:
                        sources.write(re.sub(line[10:15], Samples, line))
                    elif "LAT" in line:
                        sources.write(re.sub(line[12:15], Latitude, line))
                    elif "LON" in line:
                        sources.write(re.sub(line[12:15], Longitude, line))
                    else:
                        sources.write(re.sub(line, line, line))
    
    
    data_to_process = glob.glob(os.path.join(folder, '*.grd'))
    for data_path in data_to_process:
        raster_dataset = gdal.Open(data_path, gdal.GA_ReadOnly)
        raster = gdal.Translate(os.path.join(folder, os.path.basename(data_path) + '.tiff'), raster_dataset, format = 'Gtiff')
    
    #data_to_process = glob.glob(os.path.join(input_path, '*.grd'))
     #   for data_path in data_to_process:
    #raster_dataset = gdal.Open(data_path, gdal.GA_ReadOnly)
    #raster = gdal.Translate(os.path.join(output_path, os.path.basename(data_path)+'.tiff'), raster_dataset, format = 'Gtiff')                    
    
    print(".tiffs have been created")
    return



In [67]:
input = '/Volumes/JT/projects/uavsar/jemez/input'
test_data = '/Volumes/JT/projects/uavsar/jemez/test'
jemez_VV = '/Volumes/JT/projects/uavsar/jemez/raw_data/02122020_02192020/VV'
#uavsar_tiff_convert(test_data)
uavsar_tiff_convert(input)

7795

4713

36.121778400000004  
-106.566746880000011
.tiffs have been created


In [31]:
#for file in os.listdir("/Volumes/JT/projects/uavsar/jemez/test2"):
 #   if file.endswith(".grd"):
  #      print(os.path.join("/Volumes/JT/projects/uavsar/jemez/test2", file))

SyntaxError: invalid syntax (<ipython-input-31-2794b5587115>, line 1)

In [ ]:
#in_directory = r'/Volumes/JT/projects/uavsar/jemez/test2'
#files_to_process = glob.glob(os.path.join(in_directory, '*.grf')
#for data_path in files_to_process:
    raster_dataset = gdal.Open(data_path, gdal.GA_ReadOnly)
    #do your processing on the raster dataset here

In [63]:
input_path = '/Volumes/JT/projects/uavsar/jemez/path'
output_path = '/Volumes/JT/projects/uavsar/jemez/output'

data_to_process = glob.glob(os.path.join(input_path, '*.grd'))
for data_path in data_to_process:
    raster_dataset = gdal.Open(input_path, gdal.GA_ReadOnly)
    raster = gdal.Translate(os.path.join(path, os.path.basename(data_path)+'.tiff'), raster_dataset, format = 'Gtiff')

ValueError: Received a NULL pointer.

In [59]:
data_to_process = glob.glob(os.path.join(folder, '*.grd'))
for data_path in data_to_process:
    raster_dataset = gdal.Open(folder, gdal.GA_ReadOnly)
    raster = gdal.Translate(os.path.join(folder, os.path.basename(folder) + '.tiff'), raster_dataset, format = 'Gtiff')

ValueError: Received a NULL pointer.

In [ ]:
in_path = 'C:/GIS/Sample.tif' #input composite raster
out_path = 'C:/GIS/Output/' #output directory for individual bands as files

#Open existing raster ds
src_ds = gdal.Open(in_path)

for i in range(1,src_ds.RasterCount +1): #Save bands as individual files
    out_ds = gdal.Translate(out_path + 'band' + str(i) + '.tiff', src_ds, format='GTiff', bandList=[i])
    out_ds=None